In [13]:
import matplotlib.pyplot as plt
import os
import random
import cv2
import numpy as np


path = r"C:\Users\sonbx\OneDrive\Documents\SOCOFing\SOCOFing"
print("Path to dataset files:", path)

In [1]:
image_dir_altered = r"C:\Users\sonbx\OneDrive\Documents\SOCOFing\SOCOFing\Altered"
image_dir_real = r"C:\Users\sonbx\OneDrive\Documents\SOCOFing\SOCOFing\Real"

all_images = [f for f in os.listdir(image_dir_real) if f.endswith(('.BMP'))]

selected_images = random.sample(all_images, 10)

def process_and_plot(image_path, image_name):
    # Load the fingerprint image
    image = cv2.imread(image_path)

    # Resizing images
    resized_image = cv2.resize(image, (0, 0), fx=7, fy=7, interpolation=cv2.INTER_CUBIC)

    # Denoising
    denoised_image = cv2.fastNlMeansDenoising(resized_image, None, 7, 7, 21)

    # Convert to grayscale
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Working from here

    # Blurring
    blurred_image = cv2.GaussianBlur(enhanced_image, (3, 3), 0)


    #Adaptive threshold
    # Adaptive thresholding
    binary_image = cv2.adaptiveThreshold(enhanced_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)


    # Edge

    # Edge detection
    edges = cv2.Canny(binary_image, threshold1=60, threshold2=120)

    # Use gradient-based methods to compute orientation fields
    # Compute orientation fields for enhanced and binary images
    gradient_x = cv2.Sobel(enhanced_image, cv2.CV_64F, 1, 0, ksize=5)
    gradient_y = cv2.Sobel(enhanced_image, cv2.CV_64F, 0, 1, ksize=5)
    orientation_enhanced = cv2.phase(gradient_x, gradient_y, angleInDegrees=True)

    orientation_enhanced_normalized = cv2.normalize(orientation_enhanced, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    gradient_x = cv2.Sobel(binary_image, cv2.CV_64F, 1, 0, ksize=5)
    gradient_y = cv2.Sobel(binary_image, cv2.CV_64F, 0, 1, ksize=5)
    orientation_binary = cv2.phase(gradient_x, gradient_y, angleInDegrees=True)

    # Minutiae extraction
    minutiae_visualization = np.dstack((binary_image, binary_image, binary_image))  # Create an RGB visualization
    minutiae_points = []  # List to store minutiae (ridge endings and bifurcations)

    # Scan through the binary image and identify minutiae points
    rows, cols = binary_image.shape
    for y in range(1, rows - 1):
        for x in range(1, cols - 1):
            if binary_image[y, x] == 255:  # If the pixel is part of a ridge
                # Extract the 3x3 neighborhood
                neighborhood = binary_image[y - 1:y + 2, x - 1:x + 2]
                ridge_count = np.sum(neighborhood == 255)

                if ridge_count == 2:  # Ridge ending
                    minutiae_points.append((x, y))
                    cv2.circle(minutiae_visualization, (x, y), 3, (0, 0, 255), -1)  # Red circle for ridge ending
                elif ridge_count == 4:  # Bifurcation
                    minutiae_points.append((x, y))
                    cv2.circle(minutiae_visualization, (x, y), 3, (0, 255, 0), -1)  # Green circle for bifurcation

    # Save minutiae visualization
    output_dir = "Minutiae_processed_images"
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_minutiae_visualization.jpg"), minutiae_visualization)

    # Save other processed images
    output_dir = "Enhanced_processed_images"
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_enhanced.jpg"), enhanced_image)


    gradient_x = cv2.Sobel(binary_image, cv2.CV_64F, 1, 0, ksize=5)
    gradient_y = cv2.Sobel(binary_image, cv2.CV_64F, 0, 1, ksize=5)
    orientation_binary = cv2.phase(gradient_x, gradient_y, angleInDegrees=True)

    orientation_binary_normalized = cv2.normalize(orientation_binary, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    output_dir = "Binary_processed_images"
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_binary.jpg"), binary_image)


    # Plot the results
    fig, axes = plt.subplots(1, 8, figsize=(15, 5))
    fig, axes = plt.subplots(1, 9, figsize=(18, 5))

    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    axes[1].imshow(gray_image, cmap="gray")
    axes[1].set_title("Grayscale Image")
    axes[1].axis("off")

    axes[2].imshow(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))
    axes[2].set_title("Blurred Image")
    axes[2].axis("off")


    axes[3].imshow(edges, cmap="gray")
    axes[3].set_title("Edges")
    axes[3].axis("off")

    axes[4].imshow(enhanced_image, cmap="gray")
    axes[4].set_title("Enhanced Image")
    axes[4].axis("off")


    axes[5].imshow(orientation_enhanced, cmap="gray")
    axes[5].set_title("Orientation Enhanced")
    axes[5].axis("off")

    axes[6].imshow(binary_image, cmap="gray")
    axes[6].set_title("Binary Image")
    axes[6].axis("off")

    axes[7].imshow(orientation_binary, cmap="gray")
    axes[7].set_title("Orientation Binary")
    axes[7].axis("off")


    axes[8].imshow(cv2.cvtColor(minutiae_visualization, cv2.COLOR_BGR2RGB))
    axes[8].set_title("Minutiae Points")
    axes[8].axis("off")


    fig.suptitle(image_name, fontsize=14, y=0.02)
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.show()




for img_name in selected_images:
    img_path = os.path.join(image_dir_real, img_name)
    process_and_plot(img_path, img_name)


Input directory 'Dataset/Real' exists. Processing...
Processing directory: Dataset/Real
Number of images found: 0
No images found in directory: Dataset/Real. Skipping...
Processing directory: Dataset/Real\Person_1
Number of images found: 0
No images found in directory: Dataset/Real\Person_1. Skipping...
Processing directory: Dataset/Real\Person_1\Left_index
Number of images found: 1
[1/1] Processed and saved: output\Person_1\Left_index\1__M_Left_index_finger.BMP
Processing directory: Dataset/Real\Person_1\Left_little
Number of images found: 1
[1/1] Processed and saved: output\Person_1\Left_little\1__M_Left_little_finger.BMP
Processing directory: Dataset/Real\Person_1\Left_middle
Number of images found: 1
[1/1] Processed and saved: output\Person_1\Left_middle\1__M_Left_middle_finger.BMP
Processing directory: Dataset/Real\Person_1\Left_ring
Number of images found: 1
[1/1] Processed and saved: output\Person_1\Left_ring\1__M_Left_ring_finger.BMP
Processing directory: Dataset/Real\Person_1\

In [15]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Input dataset directory
dataset_dir = "Dataset"  # Path to your dataset

# Initialize lists for images and labels
images = []
labels = []

print("Loading images from dataset...")

# Traverse the dataset directory
for person_folder in os.listdir(dataset_dir):
    person_path = os.path.join(dataset_dir, person_folder)
    
    # Skip non-directory entries
    if not os.path.isdir(person_path):
        continue
    
    # Extract person ID from folder name (e.g., Person_1 -> 1)
    person_id = int(person_folder.split('_')[1])
    
    # Traverse subfolders (Easy, Medium, etc.)
    for difficulty_level in os.listdir(person_path):
        difficulty_path = os.path.join(person_path, difficulty_level)
        
        if not os.path.isdir(difficulty_path):
            continue
        
        # Traverse fingers (e.g., Left_index_finger)
        for finger in os.listdir(difficulty_path):
            finger_path = os.path.join(difficulty_path, finger)
            
            if not os.path.isdir(finger_path):
                continue
            
            # Traverse images in the folder
            for file in os.listdir(finger_path):
                if file.lower().endswith(('.bmp', '.png', '.jpg')):
                    image_path = os.path.join(finger_path, file)
                    
                    # Read image as grayscale
                    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        # Resize image to fixed dimensions (e.g., 128x128)
                        img_resized = cv2.resize(img, (128, 128))
                        # Flatten the image into a 1D vector
                        images.append(img_resized.flatten())
                        # Add corresponding label (person ID)
                        labels.append(person_id)

# Convert to NumPy arrays
X = np.array(images)
y = np.array(labels)

print(f"Loaded {X.shape[0]} images with {len(np.unique(y))} unique labels (persons).")

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

# Train SVM model
print("Training SVM model...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)  # Linear kernel
svm_model.fit(X_train, y_train)

# Make predictions
print("Evaluating model...")
y_pred = svm_model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()


Loading images from dataset...


IndexError: list index out of range